<a href="https://colab.research.google.com/github/shivangi-jodbhavi/face_authentication/blob/master/face_authentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import os
import time
from datetime import datetime
from uuid import uuid4

import cv2
import insightface
import matplotlib.pyplot as plt
import numpy as np

# Import necessary libraries
from flask import Flask, Response, render_template, request
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
from numpy.lib.npyio import save
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

cwd = os.path.dirname(os.path.abspath(__file__))
print('Setting current working directory to:', cwd)
os.chdir(cwd)

app = Flask(__name__, template_folder='./templates')


TRAIN_EMBEDDINGS_PATH = r'project\embeddings\train'
NEW_USER_EMBEDDINGS_PATH = r'project\embeddings\new_users'

TRAIN_IMAGES_PATH = r'project\images\train'
TEST_IMAGES_PATH = r'project\images\test'
NEW_USER_IMAGES_PATH = r'project\images\new_users'

NUM_FRAMES_FOR_AUTHENTICATION = 20
MAX_NEW_USER_IMAGES = 5

FPS_MAX = 10  # not less than 5
H = 1280
W = 720
SCALE = 1  # not more than 3


def visualize_results(image, boxes, identities, scores):
    image = np.uint8(image)
    boxes = np.array(boxes, dtype=np.int32)

    for box, identity, score in zip(boxes, identities, scores):
        text = '{} | {:.2f}'.format(identity, score)
        text_orig = (box[0] + 5, box[1] - 6)
        image = cv2.putText(
            image,
            text,
            text_orig,
            cv2.FONT_HERSHEY_COMPLEX_SMALL,
            .45, [0, 0, 0],
            4,
            lineType=cv2.LINE_AA)
        image = cv2.putText(
            image,
            text,
            text_orig,
            cv2.FONT_HERSHEY_COMPLEX_SMALL,
            .45, [255, 255, 255],
            1,
            lineType=cv2.LINE_AA)
        image = cv2.rectangle(
            image, (box[0], box[1]),
            (box[2], box[3]),
            [255, 0, 0], 1)
    return image


def imshow(image, figsize=(16, 9), mode=None):
    if mode == 'bgr':
        image = image[:, :, ::-1]

    plt.figure(figsize=figsize)
    plt.axis('off')
    plt.imshow(np.uint8(image))
    plt.show()



def get_randon_identifier():
    return str(uuid4())

def read_image(path):
    return cv2.imread(path)


def crop_image(image, box):
    x1, y1, x2, y2 = np.int32(box)
    cropped_image = image[y1:y2, x1:x2, :]
    return cropped_image


def get_embeddings(image):
    boxes = []
    embeddings = []
    faces = detection.get(image)
    for face in faces:
        embeddings.append(recognition.get(image, face))
        boxes.append(face['bbox'])
    embeddings = np.array(embeddings)
    boxes = np.array(boxes)
    return embeddings, boxes


def run(knn, image, index_to_identity):
    embeddings, boxes = get_embeddings(image)
    probs = knn.predict_proba(embeddings)
    labels = np.argmax(probs, axis=-1)
    scores = np.max(probs, axis=-1)
    identities = [index_to_identity[label] for label in labels]

    for i, score in enumerate(scores):
        if score > 0.9:
            continue
        identities[i] = 'UNKNOWN'

    viz_image = visualize_results(image.copy(), boxes, identities, scores)
    return viz_image, identities, boxes


def get_models():
    detection = FaceAnalysis(allowed_modules=['detection'])
    recognition = insightface.model_zoo.get_model(r'project\webface_r50.onnx')

    detection.prepare(ctx_id=0, det_size=(640, 640))
    recognition.prepare(ctx_id=0)
    return detection, recognition


detection, recognition = get_models()

def dump_train_embeddings():
    train_images = glob.glob(os.path.join(TRAIN_IMAGES_PATH, '*'))

    for folder in train_images:
        for img_path in glob.glob(os.path.join(folder, '*')):
            img = cv2.imread(img_path)
            faces = detection.get(img)
            feature = recognition.get(img, faces[0])

            person_name = os.path.basename(folder)
            image_name = os.path.basename(img_path).split('.')[0]
            output_path = os.path.join(
                TRAIN_EMBEDDINGS_PATH, person_name, image_name + '.npy')

            os.makedirs(
                os.path.join(TRAIN_EMBEDDINGS_PATH, person_name),
                exist_ok=True)
            np.save(output_path, feature)


def dump_new_user_embedding(img, person_name, idx):
    faces = detection.get(img)
    feature = recognition.get(img, faces[0])

    output_path = os.path.join(
        NEW_USER_EMBEDDINGS_PATH, person_name, str(idx) + '.npy')

    os.makedirs(
        os.path.join(NEW_USER_EMBEDDINGS_PATH, person_name),
        exist_ok=True)
    np.save(output_path, feature)


def train():
    train_embeddings = glob.glob(os.path.join(TRAIN_EMBEDDINGS_PATH, '*'))
    new_user_embeddings = glob.glob(os.path.join(NEW_USER_EMBEDDINGS_PATH, '*'))

    print('Found {} train identities'.format(len(train_embeddings)))
    print('Found {} new identities'.format(len(new_user_embeddings)))

    identities = []
    features = []
    for folder in train_embeddings + new_user_embeddings:
        person = os.path.basename(folder)
        folder = os.path.join(folder, '*')
        for path in glob.glob(folder):
            assert path.endswith('.npy')
            features.append(np.load(path))
            identities.append(person)

    features = np.array(features)
    identities = np.array(identities)

    #
    labels = sorted(list(set(identities)))
    index_to_identity = {i: person for i, person in enumerate(labels)}
    #
    knn = KNeighborsClassifier(n_neighbors=3)
    print(features.shape, identities.shape)
    knn.fit(features, identities)
    return knn, labels, index_to_identity


def video_loop():
    global knn, labels, index_to_identity

    new_user = None
    num_frames_seen = {}
    authenticated = []

    done = False
    while (authenticated == [] or not done):
        ret, image = video.read()
        image = np.float32(image)

        ts = time.time()
        timestamp = datetime.fromtimestamp(ts).strftime('%H:%M:%S %Y-%m-%d')

        viz_image, identities, boxes = run(knn, image, index_to_identity)

        max_area = 0
        identity_idx = 0
        for idx, box in enumerate(boxes):
            x1, y1, x2, y2 = box 
            area = (x2 - x1) * (y2 - y1)
            if area > max_area:
                max_area = area
                identity_idx = idx

        if len(identities) > 1:
            print('Multiple faces detected, only one user can be authenticated')

        identity = identities[identity_idx]

        if identity in labels:
            if identity in num_frames_seen:
                num_frames_seen[identity] += 1
                if num_frames_seen[identity] >= NUM_FRAMES_FOR_AUTHENTICATION:
                    if identity not in authenticated:
                        authenticated.append(identity)
                        print('{} authenticated at {}'.format(
                            identity, timestamp))
                        done = True
            else:
                num_frames_seen[identity] = 1

        else:
            if new_user is None:
                new_user = input(
                    'Unknown user detected, please enter user name for new user: ')
                new_user += '_{}'.format(get_randon_identifier())

            save_path = os.path.join(NEW_USER_EMBEDDINGS_PATH, new_user)
            print('Saving embeddings for {} in {}'.format(new_user, save_path))
            if not os.path.exists(save_path):
                os.makedirs(save_path, exist_ok=True)

            num_user_images = len(os.listdir(save_path))
            if num_user_images >= MAX_NEW_USER_IMAGES:
                knn, labels, index_to_identity = train()
                print('Successfull registered:', new_user)
            dump_new_user_embedding(image, new_user, num_user_images + 1)

        ret, buffer = cv2.imencode('.jpg', viz_image)
        viz_image = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + viz_image + b'\r\n')  # concat frame one by one and show result

    video.release()
    cv2.destroyAllWindows()


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/video_feed')
def video_feed():
    return Response(video_loop(), mimetype='multipart/x-mixed-replace; boundary=frame')


if __name__ == "__main__":
    if not os.path.exists(TRAIN_EMBEDDINGS_PATH):
        os.makedirs(TRAIN_EMBEDDINGS_PATH, exist_ok=True)

    if not os.path.exists(NEW_USER_EMBEDDINGS_PATH):
        os.makedirs(NEW_USER_EMBEDDINGS_PATH, exist_ok=True)

    train_identities = glob.glob(os.path.join(TRAIN_EMBEDDINGS_PATH, '*'))
    if not train_identities:
        print('Generating training embeddings')
        dump_train_embeddings()

    knn, labels, index_to_identity = train()

    video = cv2.VideoCapture(0 + cv2.CAP_DSHOW)
    video.set(cv2.CAP_PROP_FRAME_WIDTH,  H // SCALE)
    video.set(cv2.CAP_PROP_FRAME_HEIGHT, W // SCALE)
    video.set(cv2.CAP_PROP_FPS, FPS_MAX)
    video.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))

    app.run(debug=False)
